In [1]:
!pip install gTTS pydub
!pip install pydub numpy

In [2]:
from gtts import gTTS
from pydub import AudioSegment
from pydub.generators import Sine
import os

# --- Step 1: Generate "Pull" and "Rest" voice clips using gTTS ---
def create_tts(phrase, filename):
    tts = gTTS(text=phrase, lang='en', tld='com.au')  # AU = clearer female voice
    tts.save(filename)

if not os.path.exists("pull.mp3"):
    create_tts("Pull", "pull.mp3")
if not os.path.exists("rest.mp3"):
    create_tts("Rest", "rest.mp3")

pull_voice = AudioSegment.from_file("pull.mp3")
rest_voice = AudioSegment.from_file("rest.mp3")

# --- Step 2: Create high-energy background beat ---
beat = AudioSegment.silent(duration=0)
for i in range(0, 300 * 1000, 250):  # every 250 ms
    tone = Sine(880).to_audio_segment(duration=100).apply_gain(-12)
    beat += tone + AudioSegment.silent(duration=150)

# --- Step 3: Assemble track alternating pull/rest every 5 sec ---
final_track = AudioSegment.silent(duration=0)
for i in range(0, 300, 5):  # every 5 seconds for 5 minutes
    cue = pull_voice if (i // 10) % 2 == 0 else rest_voice
    segment = AudioSegment.silent(duration=0)
    segment += cue + AudioSegment.silent(duration=5000 - len(cue))
    final_track += segment

# --- Step 4: Add beat under voice cues ---
final_with_beat = final_track.overlay(beat[:len(final_track)])

# --- Step 5: Export final file ---
output_path = "pull_rest_with_voice.mp3"
final_with_beat.export(output_path, format="mp3")
print(f"Track saved as: {output_path}")


Track saved as: pull_rest_with_voice.mp3
